In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma

In [4]:
from dotenv import  load_dotenv
load_dotenv()


True

In [5]:
import pandas as pd
books=pd.read_csv('books_cleaned.csv')

In [10]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [9]:
books["tagged_description"].to_csv("tagged_description.txt", index=False, header=False)
with open("tagged_description.txt", "w", encoding="utf-8") as f:
    for desc in books["tagged_description"]:
        f.write(str(desc) + "\n")


In [11]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("tagged_description.txt", encoding="utf-8")
raw_documents = loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1,chunk_overlap=0,separator="\n")
documents=text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 479, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 284, which is longer than the specified 1
Created a chunk of size 193, which is longer than the specified 1
Created a chunk of size 877, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create

In [13]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
import os
load_dotenv()
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db_books =Chroma.from_documents(documents,
                                embedding=embedding,
                                )

In [17]:
query="A book for children"
docs=db_books.similarity_search(query,k=10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content="9781406934854 This is a reproduction of the original artefact. Generally these books are created from careful scans of the original. This allows us to preserve the book accurately and present it in the way the author intended. Since the original versions are generally quite old, there may occasionally be certain imperfections within these reproductions. We're happy to make these classics available again for future generations to enjoy!"),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9781406904833 This is a reproduction of the original artefact. Generally these books are created from careful scans of the original. This allows us to preserve the book accurately and present it in the way the author intended. Since the original versions are generally quite old, there may occasionally be certain imperfections within these reproductions. We're happy to make these classics available again for futu

In [18]:
books[books["isbn13"]==int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
4464,9781406934854,1406934852,A Commentary Upon the Gospel According to S. Luke,Saint Cyril (patriarch of Alexandria),History,http://books.google.com/books/content?id=Mclsy...,This is a reproduction of the original artefac...,2019.0,3.59,362.0,669.0,A Commentary Upon the Gospel According to S. Luke,9781406934854 This is a reproduction of the or...


In [19]:
def retrive_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)
    books_list = []

    for i in range(len(recs)):
        books_list.append(int(recs[i].page_content.split()[0].strip()))

    filtered_books = books[books["isbn13"].isin(books_list)]
    return filtered_books.head(top_k)

In [20]:
retrive_semantic_recommendations("a book for children")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
334,9780060976118,006097611X,Operation Wandering Soul,Richard Powers,Fiction,http://books.google.com/books/content?id=nIGIm...,"Highly imaginative and emotionally powerful, t...",1994.0,3.62,352.0,366.0,Operation Wandering Soul,9780060976118 Highly imaginative and emotional...
823,9780142407929,0142407925,The BFG,Roald Dahl;David Wood;Jane Walmsley,Juvenile Nonfiction,http://books.google.com/books/content?id=aDf-O...,"With notes on staging, props, and costumes, a ...",2007.0,4.26,128.0,452.0,The BFG: A Set of Plays,"9780142407929 With notes on staging, props, an..."
845,9780143036241,0143036246,The Christians and the Fall of Rome,Edward Gibbon,History,http://books.google.com/books/content?id=DAeWA...,Penguin brings you the works of the great thin...,2005.0,3.57,96.0,202.0,The Christians and the Fall of Rome,9780143036241 Penguin brings you the works of ...
872,9780143039853,0143039857,The Outsiders,S. E. Hinton;Jodi Picoult,Fiction,http://books.google.com/books/content?id=9PyIP...,The struggle of three brothers to stay togethe...,1967.0,4.08,160.0,1558.0,The Outsiders,9780143039853 The struggle of three brothers t...
965,9780192833600,019283360X,The Canterbury Tales,Geoffrey Chaucer;David Wright,Poetry,http://books.google.com/books/content?id=5WkDQ...,Provides a modern verse translation of the sto...,1998.0,3.49,465.0,249.0,The Canterbury Tales,9780192833600 Provides a modern verse translat...
975,9780192836380,0192836382,La Dame Aux Camélias,Alexandre Dumas;David Coward,Fiction,http://books.google.com/books/content?id=Aqft1...,This new translation successfully combines a f...,1986.0,3.99,256.0,14702.0,La Dame Aux Camélias,9780192836380 This new translation successfull...
990,9780192854520,0192854526,Augustine: A Very Short Introduction,Henry Chadwick,Biography & Autobiography,http://books.google.com/books/content?id=QnrRB...,"Very Short Introductions offer stimulating, ac...",2001.0,3.74,144.0,219.0,Augustine: A Very Short Introduction,9780192854520 Very Short Introductions offer s...
1169,9780310929550,0310929555,Zondervan NIV Study Bible,Kenneth L. Barker;Donald W. Burdick,Bibles,http://books.google.com/books/content?id=WvYLG...,#1 Best-selling study Bible in the best-sellin...,2002.0,4.70,2198.0,4092.0,Zondervan NIV Study Bible: New International V...,9780310929550 #1 Best-selling study Bible in t...
1219,9780312313678,0312313675,The New York Times Guide to Essential Knowledge,The New York Times,Reference,http://books.google.com/books/content?id=zqkdN...,"Presents information on such subjects as art, ...",2004.0,4.09,1104.0,150.0,The New York Times Guide to Essential Knowledg...,9780312313678 Presents information on such sub...
1288,9780312890216,0312890214,The Starry Rift,James Tiptree,Fiction,NaN,This novel set in the far-future and filled wi...,1994.0,3.82,250.0,220.0,The Starry Rift,9780312890216 This novel set in the far-future...


In [21]:
retrive_semantic_recommendations("a book for grown man")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
233,9780060790592,0060790598,Mysteries of Pittsburgh,Michael Chabon,Fiction,http://books.google.com/books/content?id=wh8B4...,By the Pulitzer Prize-winning author of The Am...,2005.0,3.64,320.0,21338.0,Mysteries of Pittsburgh: A Novel,9780060790592 By the Pulitzer Prize-winning au...
291,9780060924980,0060924985,The Infinite Plan,Isabel Allende,Fiction,http://books.google.com/books/content?id=80pDR...,"Selling more than 65,000 copies and topping be...",1994.0,3.71,384.0,7102.0,The Infinite Plan: A Novel,"9780060924980 Selling more than 65,000 copies ..."
387,9780061177590,0061177598,Women,Charles Bukowski,Fiction,http://books.google.com/books/content?id=170LA...,Tells the story of an ugly middle-aged man who...,2007.0,3.87,291.0,49265.0,Women,9780061177590 Tells the story of an ugly middl...
512,9780099474128,0099474123,Women Who Love Too Much,Robin Norwood,Alcoholics,http://books.google.com/books/content?id=_v6tu...,Is having 'somebody to love' the most importan...,2004.0,4.00,314.0,239.0,Women Who Love Too Much,9780099474128 Is having 'somebody to love' the...
544,9780130235015,0130235016,Things Fall Apart,Chinua Achebe;Sandra Widner,Igbo (African people),NaN,"[This book is] a simple story of a ""strong man...",2000.0,3.64,113.0,40.0,Things Fall Apart: An Adapted Classic,9780130235015 [This book is] a simple story of...
560,9780140110876,0140110879,You Bright and Risen Angels,William T. Vollmann,Fiction,http://books.google.com/books/content?id=d0buA...,This comic and surreal novel about the beastli...,1988.0,4.08,635.0,747.0,You Bright and Risen Angels: A Cartoon,9780140110876 This comic and surreal novel abo...
749,9780141184289,0141184280,The Good Soldier Svejk,Jaroslav Hašek,Czechoslovakia,http://books.google.com/books/content?id=lRXvO...,This text is the classic novel of the little m...,2000.0,4.11,752.0,84.0,The Good Soldier Svejk,9780141184289 This text is the classic novel o...
872,9780143039853,0143039857,The Outsiders,S. E. Hinton;Jodi Picoult,Fiction,http://books.google.com/books/content?id=9PyIP...,The struggle of three brothers to stay togethe...,1967.0,4.08,160.0,1558.0,The Outsiders,9780143039853 The struggle of three brothers t...
975,9780192836380,0192836382,La Dame Aux Camélias,Alexandre Dumas;David Coward,Fiction,http://books.google.com/books/content?id=Aqft1...,This new translation successfully combines a f...,1986.0,3.99,256.0,14702.0,La Dame Aux Camélias,9780192836380 This new translation successfull...
1169,9780310929550,0310929555,Zondervan NIV Study Bible,Kenneth L. Barker;Donald W. Burdick,Bibles,http://books.google.com/books/content?id=WvYLG...,#1 Best-selling study Bible in the best-sellin...,2002.0,4.70,2198.0,4092.0,Zondervan NIV Study Bible: New International V...,9780310929550 #1 Best-selling study Bible in t...
